In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from scipy.interpolate import interp1d

In [8]:
df_demo_radar = pd.read_csv(r'../data/demo_radar.csv')

In [13]:
def new_col_5_bins (data, column_name : str, new_column: str):
    # Assign boundaries
    boundary_1 = data[column_name].quantile(0.2) 
    boundary_2 = data[column_name].quantile(0.4) 
    boundary_3 = data[column_name].quantile(0.6) 
    boundary_4 = data[column_name].quantile(0.8) 
    boundary_5 = data[column_name].max()
    
    # Define a function to map the values to their respective bins
    def map_to_bin(value):
        if value <= boundary_1:
            return 1
        elif value <= boundary_2:
            return 2
        elif value <= boundary_3:
            return 3
        elif value <= boundary_4:
            return 4
        else:
            return 5

    # Apply the function to the column and assign the result to the new column
    data[new_column] = data[column_name].apply(map_to_bin)

    return data

In [11]:
df_demo_copy = df_demo

In [24]:
df_demo_radar = new_col_5_bins(df_demo_copy, 'logons_6_mnth', 'radar_digital')

In [9]:
# Segment by gendr

demo_m = pd.DataFrame(df_demo_radar[df_demo_radar['gendr'] == 'M']).reset_index(drop=True)
demo_f = pd.DataFrame(df_demo_radar[df_demo_radar['gendr'] == 'F']).reset_index(drop=True)
demo_u = pd.DataFrame(df_demo_radar[df_demo_radar['gendr'] == 'U']).reset_index(drop=True)
demo_x = pd.DataFrame(df_demo_radar[df_demo_radar['gendr'] == 'X']).reset_index(drop=True)

In [10]:
m_radar = list(demo_m[list(demo_m.columns)[-5:]].mean())
f_radar = list(demo_f[list(demo_f.columns)[-5:]].mean())
u_radar = list(demo_u[list(demo_u.columns)[-5:]].mean())
x_radar = list(demo_x[list(demo_x.columns)[-5:]].mean())

In [11]:
def interpolator (original_values : list, all_values, new_range_min, new_range_max):
    m = interp1d([min(all_values),max(all_values)],[new_range_min,new_range_max])
    new_values = list(m(original_values))
    return new_values

In [14]:
m_radar_int = interpolator(m_radar, radar_values, 1,5)
f_radar_int = interpolator(f_radar, radar_values, 1,5)
u_radar_int = interpolator(u_radar, radar_values, 1,5)
x_radar_int = interpolator(x_radar, radar_values, 1,5)

In [ ]:
f_radar_int = list(m(f_radar))
u_radar_int = list(m(u_radar))
x_radar_int = list(m(x_radar))

In [44]:
df_demo_radar.to_csv('demo_radar', sep = ',')

In [5]:
df_demo_radar.head(10)

,Unnamed: 0.1,Unnamed: 0,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,gendr_v,radar_tenure,radar_age,radar_bal,radar_demand,radar_digital
0,0,0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0,0,1,4,2,4,5
1,1,1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0,0,2,4,4,4,5
2,2,2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0,0,1,2,3,4,5
3,3,3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0,1,4,3,3,3,3
4,4,4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0,2,3,2,4,1,1
5,5,5,3727881,5.0,71.0,30.5,U,2.0,23915.60,0.0,3.0,0,1,1,1,1,1
6,6,6,272934,5.0,66.0,58.5,U,2.0,27021.42,2.0,5.0,0,1,4,1,2,2
7,7,7,388801,30.0,361.0,57.5,M,5.0,522498.72,1.0,4.0,1,5,4,5,1,2
8,8,8,285619,30.0,369.0,67.5,M,2.0,299388.72,3.0,6.0,1,5,5,5,3,3
9,9,9,8198645,15.0,189.0,54.5,F,2.0,382303.83,6.0,9.0,2,4,4,5,4,5


In [13]:
radar_values = m_radar + f_radar + u_radar + x_radar

In [22]:
m_radar_int

[5.0,
 4.3219845302436966,
 4.596556522864818,
 3.9799669892229517,
 4.104825398880222]

In [17]:
f_radar

[3.586696562032885,
 3.254814033236613,
 3.0697265453266507,
 2.5940824760397434,
 2.6477622439110173]

In [18]:
u_radar

[1.5308846695962193,
 2.5385125611475003,
 2.598001824061023,
 2.730204792305779,
 2.7944200315065086]

In [17]:
categories = ['Tenure','Age','Wealth',
              'Demanding', 'Digitally active']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=m_radar_int,
      theta=categories,
      fill='toself',
      name='Male'
))
fig.add_trace(go.Scatterpolar(
      r=f_radar_int,
      theta=categories,
      fill='toself',
      name='Female'
))

fig.add_trace(go.Scatterpolar(
      r=u_radar_int,
      theta=categories,
      fill='toself',
      name='Unknown'
))

# fig.add_trace(go.Scatterpolar(
#       r=x_radar_int,
#       theta=categories,
#       fill='toself',
#       name='X'
# ))

fig.update_layout(
    margin=dict(l=1,r=1,b=20,t=10),
    polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False,font=dict(
    family="Arial, sans-serif",size = 20 
))

pio.write_image(fig, 'radar10.svg', format='svg', scale=10) 

In [7]:
import plotly.express as px

categories = ['Tenure','Age','Wealth',
              'Demanding', 'Digitally savvy']

df = pd.DataFrame(dict(
    r=m_radar_int,
    theta=categories))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.update_traces(fill='toself')
fig.write_image("radar.png")